# Introduction
## Background
Have you ever tried to use RPG Maker autotiles/terrain tiles in TileD or another mapping tool?

It doesn't work great.

The reason is because RPG Maker's autotiles are actually subdivided into 4 'mini-tiles' which are composited back together into full-sized tiles as you paint.

TileD can approximate this, but doesn't fully understand where each part of each tile goes. As such, it picks randomly for each of the edge and center peices, and doesn't create perfect patterns. If you already noticed this, well, that's probably why you're here.

## This Program
That's where this program comes in.

It's purpose is to take RPG Maker tilesets (which are 9 tiles), take them apart, composite them, and output the 47-tile tilests that work better in basically every other program.

You don't have to write any code to use it, but you do have to upload files, write in the names of those files, and change some other parameters. Then you have to go down a long list of 'code blocks' and run them with the little play buttons on the left of each block, one after the other.

If it all goes well, it'll output your finished 47-tile tilesets at the end.

## New Features
- Can now stitch together all the autotiles into combined tilesets again! No more juggling 20+ pngs!
- You can exclude tiles and have them extracted and set aside. Perfect for when rando tiles are jammed in with your autotiles.
- Can actually stitch together any documents that contain tiles that are all organized the same. This saves space and lets you combine autotiles form multiple source files.
- Code has documentation built in; if you take it and use it in your own editor, like VSCode, it's easier to find your way around

## Notes
This is a strict upgrade to the converter found [here](https://colab.research.google.com/drive/1pK-jr_15GjqxKqSUtrmQuLVsSE2_IQq_?usp=sharing) I'm leaving a link to it because it's less intimidating to anyone who's never worked with Python before, and might encourage them to try making their own tools.

# The Code
## Core Code
  Run this to setup everything you need, then navigate wayyy past it down to the ***next*** block, because that's where you're going to have to edit in your parameters.


In [3]:
from PIL import Image
import math
from typing import List

class TileSizeObject:
    """You're going to want to create one TileSizeObject for each size and layout of tileset you plan to work with. Mixing and matching tile sizes can be dangerous when trying to compact multiple tilesets into one."""

    inputRows = 3
    '''These are constants governing the dimensions of autotiles in input for RPG Maker tilsets Don't edit unless you are making dramatic changes to TilsetConverter's outputTiles.'''
    inputCols = 2
    '''These are constants governing the dimensions of autotiles in input for RPG Maker tilsets Don't edit unless you are making dramatic changes to TilsetConverter's outputTiles.'''
    ignoredTiles = 1
    '''These are constants governing the dimensions of autotiles in input for RPG Maker tilsets Don't edit unless you are making dramatic changes to TilsetConverter's outputTiles.'''

    def __init__(self, tileSize:tuple, startingPixel:tuple, terrainTypes:tuple):
        """You're going to want to create one TileSizeObject for each size and layout of tileset you plan to work with
        ### Parameters
        1. tileSize: Two integers (x,y) describing the size of a tile in pixels]
        2. startingPixel: If there is an offset or margin on the left hand side, adjust startingPixel. In most cases this will be (0,0)
        3. terrainTypes: Two integers (x,y) describing the rows and columns of autotiles/terrain types. For RPGMaker MV/MZ, this is going to be (8,4)
        """
        self.tileSize = tileSize
        '''The size of a tile (x,y) in pixels. Don't update after setting unless you also call calculateDerivedValues()'''

        self.startingPixel = startingPixel
        '''(x,y), in pixels; if there is a margin on the left or top of your tileset, you can adjust where the tile extraction process starts by setting this equal to that margin'''

        self.terrainTypes = terrainTypes
        '''How many terrains are expected per input tilesheet, defined as integers in an (x,y) tuple. Don't update after setting unless you also call calculateDerivedValues()'''

        #These are defined here entirely so I can attach tooltips

        self.totalTerrainTypes:int = None
        '''The total number of terrain types. self.terrainTypes[0] * self.terrainTypes[1]'''
        self.terrainPixelSize:tuple = None
        '''The amount of pixels taken up by each terrain. This is '''
        self.totalPixelSize:tuple = None
        '''The total number of pixels taken up by the autotiles, regardless of how large the file is.'''
        self.halfTile:int = None
        '''Because we are working with half-tiles, having a halfTile saved helps avoid unnecessary division.'''
        self.sourceTiles:List[tuple] = None
        '''A set of 4-entry tuples specifying the source locations of each of the subtiles form which the 47-tiles will be composited, in coordinates local to that specific autotile'''

        self.calculateDerivedValues()
        self.calculateTilePositions()

    def calculateDerivedValues(self):
        '''Needs to be run after the initial values are set. Why is this a separate function from init? I dunno, maybe you want to update something. Organizationally it worked well'''

        self.totalTerrainTypes = self.terrainTypes[0] * self.terrainTypes[1]
        self.terrainPixelSize = (2*self.tileSize, 3*self.tileSize)
        self.totalPixelSize = (self.terrainTypes[0] * self.terrainPixelSize[0], self.terrainTypes[1] * self.terrainPixelSize[1])

        self.halfTile = int(self.tileSize/2)


    def calculateTilePositions(self):
        '''Calculate where in each terrain the subtiles are actually located and saves that source information into the sourceTiles array, for use later by the TileConverter. See the code for an ASCII diagram of which RPGMaker tiles end up at which indecies of the array.'''

        '''
        RPG Maker Subtile Layout.
        =========================
        RPG Maker autotiles are laied out in a 2x3 grid. The upper left is actually never used!
        The rest of the tiles are broke up internally int 4 subtiles a peice, and rules about
        which tiles go where are used to composite all 47 Wang tile shapes. We're immitating
        that here. Here is a ASCII graphic demonstrating which index each of the subtiles
        will be going into in the flat array self.sourceTiles


        |-----------|-----------|
        |   X   X   |   0   1   |
        |   X   X   |   2   3   |
        |-----------|-----------|
        |   4   5   |   8   9   |
        |   6   7   |   10  11  |
        |-----------|-----------|
        |   12  13  |   16  17  |
        |   14  15  |   18  19  |
        |-----------|-----------|


        '''

        #One of RPGMaker's tiles isn't even used, so to get the number of source tiles we need to draw from, we'll multiply the rows and cols (3x2) and subtract 1
        numRegularTiles = TileSizeObject.inputRows*TileSizeObject.inputCols-TileSizeObject.ignoredTiles;

        #We're creating subtiles, so that's 4 subtiles from each 'regular' sized tile. We want to create a flat array to make accessing it easy, and so TileConverter doesn't
        #actually need to know the shape that the RPGMaker tiles take.
        self.sourceTiles = [None]*(4*numRegularTiles)


        #Since source-tiles is stored in a flat array,
        #m will keep track of where we are in the flat array
        #while k,h,j,and i track the nested arrays
        m = 0

        #Explore the vertical dimension second
        for k in range(0,3):

            #Explore the horizontal dimension first
            for h in range(0,2):

                #Just ignore that upper left square
                if (k==0) and (h == 0):
                    continue

                #explore the vertical dimension second
                for j in range(0,2):

                    #explore the horizontal dimension first
                    for i in range(0,2):

                        #Get the rectangle for where each tile starts and ends.
                        startX = i*self.halfTile + h*self.tileSize
                        endX = startX + self.halfTile
                        startY = j*self.halfTile + k*self.tileSize
                        endY = startY + self.halfTile

                        #Throw it into our source tile array
                        self.sourceTiles[m] = (startX, startY, endX, endY)
                        m +=1



class TileSetInfo:
    '''TileSetInfo records information about the tileset you want to load and edit, including it's file name. The tileset must be saved in the png format'''
    def __init__(self, fileName:str, tileSizeObject:TileSizeObject, newName:str = None):
        '''TileSetInfo records information about the tileset you want to load and edit, including it's file name. The tileset must be saved in the png format
        1. fileName: The name of the file you want to operate on. Do not include the file extension; this whole thing only works on pngs.
        2. tileSizeObject: Information about the tilset, including its resolution, dimensions, and any margin
        3. newName: If you want to completely change the name of your results, change this newName to something.
        '''
        self.fileName = fileName
        ''' The name of the file you want to operate on. Do not include the file extension. Don't modify after setting, not unless you're going to update newName and loadPath'''

        self.tileSizeObject = tileSizeObject
        '''Information about the tilset, including its resolution, dimensions, and any margin'''

        self.loadPath = fileName + ".png"
        '''The actual place we will load the file from.'''

        '''The new name you want to write the output to'''
        self.newName = newName
        if newName is None:
            self.newName = fileName



class TileSetConverter:
    '''Can be reused over and over again to convert multiple tilesets. This TilesetConverter is designed to map RPGMaker tiles to a 47-tile format, also called Wang tiles. You can edit it to map to any format, or edit the order of the output.'''
    outputRows = 6
    '''These are constants governing the dimensions of the output for the 47-tile tilesets. Don't edit unless you are also filling outputTiles yourself.'''
    outputCols = 8
    '''These are constants governing the dimensions of the output for the 47-tile tilesets. Don't edit unless you are also filling outputTiles yourself.'''

    def __init__(self, weldSize:tuple = None, asideSize:tuple = None, outputSuffix:str = "_output", extraSuffix:str = "_other"):
        '''Create a TilesetConverter, which can be reused over and over again to convert multiple tilesets. This TilesetConverter is designed to map RPGMaker tiles to a 47-tile format, also called Wang tiles. You can edit it to map to any format, or edit the order of the output.
        1. weldSize: An integer (x,y). If you want to output autotiles individually, leave this as None. If you want to put all your autotiles together in a finalized tileset, set this to the pixel dimensions of the tileset you want.
        2. asideSize: An integer (x,y). If you want to output non-autotiling, excluded tiles individually (or not at all), leave this as None. If you want to put all your autotiles together in a finalized tileset, set this to the pixel dimensions of the tileset you want.
        3. outputSuffix: When creating output files, this suffix will be appended to your autotiles
        4. extraSuffix: When creating output files, this suffix will be appended to your non-autotiles
        '''

        self.outputTiles = None
        '''Will Store a 2-dimensional integer map describing which RPGMaker subtiles are going to be mapped to which subtiles of the 47-tile Autotiler'''

        #Create your own function and run it here if you want to output to a different format
        self.fortySevenTileOutput()

        #Prepare for welding, er I mean stitchign, er, I mean compositing, oh it doesn't matter what word I'm using
        self.allAutotiles:List[Image.Image] = None
        '''A flat array of all the autotile images that have been composited from the tileset.'''

        self.allWelds:List[Image.Image] = None
        '''A flat array of all the final output tilesets built from the autotiles.'''

        self.weldSize = weldSize
        '''None, if welds aren't required, or a (x,y) tuple indicating the pixel dimensions of the output files'''

        self.outputSuffix = outputSuffix
        '''When creating output files, this suffix will be appended to your autotiles'''
        self.extraSuffix = extraSuffix
        '''When creating output files, this suffix will be appended to your non-autotiles'''

        self.asideSize = asideSize
        '''Identical to weldSide, but for things that were excluded for not being autotiles'''
        self.nonAutotiles:List[Image.Image] = None
        '''An array that stores every block of tiles that were excluded for not being autotiles'''
        self.allAsideWelds:List[Image.Image]= None
        '''An array that stores all the final output tilesets built from the non-autotiling tiles excluded from the conversion'''

        #Aside


    def zeroOutOutPutTiles(self):
        '''Use if you want to zero out the output tiles'''

        #because we are working with subtiles, there will be twice as many columns and rows
        rows = TileSetConverter.outputRows*2
        cols = TileSetConverter.outputCols*2
        self.outputTiles = [None]*rows
        for j in range(0,rows):
            self.outputTiles[j] = [None]*cols
            for i in range(0,cols):
                self.outputTiles[j][i] = 0


    def fortySevenTileOutput(self):
        '''Builds the map saved in outputTiles that maps subtiles to their final output locations.
        This is the arrangement of tiles I find most sensible, but which is not the vanilla GameMaker arrangement, so you might want to make your own version of this'''

        rows = TileSetConverter.outputRows*2
        self.outputTiles = [None]*rows

        self.outputTiles[0] = [4,5,8,9,  16,13,16,13,    4,5,8,9,    16,1,0,13]
        self.outputTiles[1] = [6,7,10,11,    10,3,2,7,   6,3,2,11,   2,3,2,3]
        self.outputTiles[2] = [12,13,16,17,  16,1,0,13,  12,1,0,17,  0,1,0,1]
        self.outputTiles[3] = [14,15,18,19,  10,7,10,7,  14,15,18,19,    10,3,2,7]

        self.outputTiles[4] = [12,13,8,5,    12,13,8,5,  12,1,8,5,   12,1,8,5]
        self.outputTiles[5] = [6,7,10,7,    6,3,2,13,   6,7,10,3,   6,3,2,3]
        self.outputTiles[6] = [16,13,16,17,  16,1,0,17,  0,13,16,17, 0,1,0,17]
        self.outputTiles[7] = [18,15,10,11,   18,15,10,11, 18,15,2,11, 18,15,2,11]

        self.outputTiles[8] = [16,13,0,13,   4,9,4,5,    8,5,8,9,    12,17,0,13]
        self.outputTiles[9] = [2,3,2,7,  6,11,14,15, 18,15,18,19,    6,11,10,3]
        self.outputTiles[10] = [16,1,0,1,    12,17,16,13,    4,9,0,1,    None, None, 16, 1]
        self.outputTiles[11] = [10,3,10,7,   14,19,10,7, 14,19,2,3,      None, None, 2,7]


    def getSavePaths(self, tileset:TileSetInfo, n:int = None) -> int:
        '''Get saved paths based on the tileset name'''
        tso = tileset.tileSizeObject
        savePaths = [None] * tso.totalTerrainTypes
        if n is None:
            n = tso.totalTerrainTypes
        for i in range(0, n):
           savePaths[i] = tileset.newName + self.outputSuffix + str(i) + ".png"
        return savePaths


    def getExtraSavePaths(self, tileset:TileSetInfo, n:int = None) -> int:
        '''Get 'extra' saved paths based on the tileset name (these are for things that are not autotiles)'''
        tso = tileset.tileSizeObject
        savePaths = [None] * tso.totalTerrainTypes
        if n is None:
            n = tso.totalTerrainTypes
        for i in range(0, n):
           savePaths[i] = tileset.newName + self.extraSuffix + str(i) + ".png"
        return savePaths


    def convert(self, tileset:TileSetInfo, toExcludeArray:list):
        '''Actually perform the conversion between the RPG Maker format and the new format'''

        #get out that size object in something quicker to type
        tso = tileset.tileSizeObject

        saveWidth = tso.tileSize*TileSetConverter.outputCols
        saveHeight = tso.tileSize*TileSetConverter.outputRows

        # Open File
        toEdit = Image.open(tileset.loadPath).convert("RGBA")

        # Create New File
        autoTileN = tso.totalTerrainTypes

        #Account for exclusions
        if toExcludeArray is not None:
            autoTileN -= len(toExcludeArray)
            self.nonAutotiles = [None] * len(toExcludeArray)
        self.allAutotiles = [None] * autoTileN

        #keep track of which tile, autotile, and non-autotile we are on
        m = 0
        mAuto = 0
        mAside = 0

        #there's a certain amount of terrain types across and down, columns and rows. let's iterate through them
        for h in range(0, tso.terrainTypes[1]):
            for k in range(0, tso.terrainTypes[0]):

                # First, we need to know if we're excluding this given tile, for not being an autotile.
                if toExcludeArray is None or not (m in toExcludeArray):
                    #Create an image to save to
                    autotile = Image.new(mode="RGBA", size=(saveWidth, saveHeight))

                    #Now we know that each of the RPGMaker tilesets is broken up into subtiles, so our output also
                    #has to be assembled from subtiles. Specifically there's 4x the amount of subtiles, 2x on each axis
                    #as there are actual output tiles
                    for j in range(0,TileSetConverter.outputRows*2):
                        for i in range(0,TileSetConverter.outputCols*2):

                            #Look up which tile is meant to be in this slot
                            which = self.outputTiles[j][i]

                            #Don't print any tile if the key is 'None' None is a totally valid key
                            if which == None:
                                continue

                            #Get the rectangle bounding the tile that's meant to be in this slot. It's [inclusive,exclusive)
                            copyBox = tso.sourceTiles[which]

                            #Adjusting that rectangle by starting pixel size and based on which terrain we're converting
                            source = (  copyBox[0] + tso.startingPixel[0] + k*tso.terrainPixelSize[0],
                                        copyBox[1] + tso.startingPixel[1] + h*tso.terrainPixelSize[1],
                                        copyBox[2] + tso.startingPixel[0] + k*tso.terrainPixelSize[0],
                                        copyBox[3] + tso.startingPixel[1] + h*tso.terrainPixelSize[1])

                            #Get the location we'll be pasting the tile to.
                            pastePoint = (i*tso.halfTile, j*tso.halfTile)

                            #Debugging
                            #print(str(pastePoint) + ", tile: " + str(which) + ", i: " + str(i) + ", j:" + str(j))

                            #Paste the tile (no, you don't just use 'paste' or 'copy', you have to use this unweildy monster)
                            autotile.alpha_composite(im=toEdit,dest=pastePoint, source=source)


                    #Add it to our array of autotiles
                    self.allAutotiles[mAuto] = autotile

                    #iterate to keep track of which autotile we're on
                    mAuto+=1

                #Something isn't an autotile!
                else:
                    #but it's occupying the same space
                    nonAutoTile = Image.new(mode="RGBA", size=(tso.terrainPixelSize[0], tso.terrainPixelSize[1]))

                    #We can just grab out the whole block, we don't have to split it into minitiles/subtiles
                    source = (  k*tso.terrainPixelSize[0] + tso.startingPixel[0],
                                h*tso.terrainPixelSize[1] + tso.startingPixel[1],
                                k*tso.terrainPixelSize[0] + tso.startingPixel[0] + tso.terrainPixelSize[0],
                                h*tso.terrainPixelSize[1] + tso.startingPixel[1] + tso.terrainPixelSize[1])

                    #print("sourcepoint: " + str(source[0]), ", " +str(source[1]), ", " +str(source[2]), ", " +str(source[3]))
                    #print("k: " + str(k) + ", h:" + str(h))

                    #likewise the pastepoint will always be the upper left hand corner, cause we're taking the whole
                    #thing at once
                    pastePoint = (0,0)
                    nonAutoTile.alpha_composite(im=toEdit, dest=pastePoint, source=source)
                    #print("Aside: " + str(mAside))

                    #Add it to our array
                    self.nonAutotiles[mAside] = nonAutoTile

                    #iterate to keep track of which non-autotile we're on
                    mAside += 1
                m += 1
        toEdit.close()

    def weldOutputs(self, tileset:TileSetInfo):
        '''Take all our autotiles and stuff them into tilesets'''

        tso = tileset.tileSizeObject

        saveWidth = tso.tileSize*TileSetConverter.outputCols
        saveHeight = tso.tileSize*TileSetConverter.outputRows

        #See how many rows and columns of autotiles will fit on each axis of the output sheet
        weldFitsPerAxis = (math.floor(self.weldSize[0] / saveWidth), math.floor(self.weldSize[1]/saveHeight))
        weldFits = weldFitsPerAxis[0] * weldFitsPerAxis[1]

        if weldFits <= 0:
            print("Output file resolution of " + self.weldSize + " too small to fit sets of size " + saveWidth + ", " + saveHeight)
            return

        #Calculate the total number of sheets we're going to need
        totalWeldN = math.ceil(len(self.allAutotiles)/weldFits)

        #Create the array to hold the tilesheets
        self.allWelds = [None]*totalWeldN

        #since all of the autotiles are in their own images, they 'source' we need for the copy/paste will
        #be the same for each autotile, starting in the upper left hand corner
        source = (0, 0, saveWidth, saveHeight)

        #Track position in the flat array allWelds
        n = 0

        #Iterate through all the tilesheets
        for m in range(0, totalWeldN):

            #Make the tilesheet
            weld = Image.new(mode="RGBA", size=(self.weldSize[0], self.weldSize[1]))

            #Iterate through the columns and rows of the tilesheet
            for j in range(0, weldFitsPerAxis[1]):
                for i in range(0, weldFitsPerAxis[0]):

                    #If we've reached the end of our list of autotiles, end this loop
                    #because we no longer need to copy/paste any to the final tileset,
                    #we're done.
                    if n >= len(self.allAutotiles): break
                    autotile = self.allAutotiles[n]

                    #We don't have to calculate the source for each, because it's the same
                    #for each autotile and calculated above, but the paste-point is
                    #changing as we move up and down and across the sheet
                    pastePoint = (i*saveWidth, j*saveHeight)

                    #do the copy paste
                    weld.alpha_composite(im=autotile,dest=pastePoint, source=source)

                    #iterate to keep tracke of what autotile we're on
                    n+=1

            #save all that work in our array
            self.allWelds[m] = weld


    def weldAside(self, tileset:TileSetInfo):
        '''Create tilsets from everything that isnt' an autotile'''
        tso = tileset.tileSizeObject

        #How many of those things fit in a tilesheet?
        weldFitsPerAxis = (math.floor(self.asideSize[0] / tso.terrainPixelSize[0]), math.floor(self.asideSize[1]/tso.terrainPixelSize[1]))
        weldFits = weldFitsPerAxis[0] * weldFitsPerAxis[1]

        #Hopefully more than 0; but it is possible that for very large blocks, you might have sent in too small a file resolution
        if weldFits <= 0:
            print("weldAside(): Output file resolution of " + self.asideSize + " too small to fit sets of size " + tso.terrainPixelSize[0] + ", " + tso.terrainPixelSize[1])
            return

        #Total number of tilsets we'll use
        totalWeldN = math.ceil(len(self.nonAutotiles)/weldFits)
        #print("totalWeldN for weldAside is: " + str(totalWeldN))

        #Create the array to store the tilesets
        self.allAsideWelds = [None]*totalWeldN

        #tso.terrainPixelSize is the size of the pre-conversion autotile, which is the same size
        #as the block of stuff that wasn't an autotile that we excluded and now need to put in these
        # 'aside' tilesets.

        #Since each non-autotiling block is in its own image object, the source is
        #pinned from the upper left hand corner
        source = (0, 0, tso.terrainPixelSize[0], tso.terrainPixelSize[1])

        #Track our position  among the 'asides', the non-autotiles
        n = 0

        #Iterate through all our tilesheets
        for m in range(0, totalWeldN):

            #Create the tilesheet
            weld = Image.new(mode="RGBA", size=(self.asideSize[0], self.asideSize[1]))

            #begin iterating through the rows and columns we've divided the tilesheet into
            for j in range(0, weldFitsPerAxis[1]):
                for i in range(0, weldFitsPerAxis[0]):

                    #If we ran out of things to put in the tilesheet, stop this loop
                    if n >= len(self.nonAutotiles):
                        break

                    #Grab the non-auto-tiling thing.
                    aside = self.nonAutotiles[n]

                    if aside is None:
                        print("error, nonAutoTiles[n] is None on Aside " + str(n))

                    #We don't need to get source here, because it's the same for every side
                    #but we do need to get the pastepoint, because that's changing as we
                    # move through the rows/columns of the output sheet.
                    pastePoint = (i*tso.terrainPixelSize[0], j*tso.terrainPixelSize[1])

                    #actually copy/paste the pixels
                    weld.alpha_composite(im=aside,dest=pastePoint, source=source)

                    #iterate through the 'asides'
                    n+=1

            #Add all that work to our list of tilsets
            self.allAsideWelds[m] = weld


    def saveAutotiles(self, tileset:TileSetInfo):
        '''Save all the autotiles individually in their own files'''

        savePaths = self.getSavePaths(tileset)
        for i in range(0, len(self.allAutotiles)):
            self.allAutotiles[i].save(fp=savePaths[i])


    def saveAsides(self, tileset:TileSetInfo):
        '''Save everything that wasn't an autotile individually in its own file'''

        savePaths = self.getExtraSavePaths(tileset)
        for i in range(0, len(self.nonAutotiles)):
            self.nonAutotiles[i].save(fp=savePaths[i])


    def convertAndSaveAutotiles(self, tileset:TileSetInfo, toExcludeArray = None, setAside=False):
        '''Converts each autotile and saves them in separate files
        1. tileset: the tileset you want converted
        2. toExcludeArray: tiles you want excluded from converstion, presumably because they are not autotiles
        3. setAside: do you want those excluded files saved and placed into their own files?
        '''

        self.convert(tileset=tileset, toExcludeArray=toExcludeArray)
        self.saveAutotiles(tileset=tileset)
        if setAside:
            self.saveAsides(tileset)

        #time to clean up
        self.cleanUp()


    def saveWelds(self, tileset:TileSetInfo):
        '''Save all the tilesets of autotiles'''

        n = len(self.allWelds)
        savePaths = self.getSavePaths(tileset, n)
        for i in range(0, n):
            self.allWelds[i].save(fp=savePaths[i])


    def saveAsideWelds(self, tileset:TileSetInfo):
        '''Save all the tilesets that weren't autotiles'''

        n = len(self.allAsideWelds)
        savePaths = self.getExtraSavePaths(tileset, n)
        for i in range(0, n):
            self.allAsideWelds[i].save(fp=savePaths[i])


    def convertWeldAndSaveAutotiles(self, tileset:TileSetInfo, toExcludeArray = None, setAside=False):
        '''Converts each autotile, but rather than saving it in isolation, instead combines the autotiles into bigger tilesheets such as 1024x1024 or 2048x2048. With 48x48 tiles, there's no perfect fit; 2048x2048 delivers the least waste.
        1. tilset: the tileset you want converted
        2. toExcludeArray: tiles you want excluded from converstion, presumably because they are not autotiles
        3. setAside: do you want those excluded files saved and placed into their own file?
        '''

        #Convert the RPG Maker autotiles to 47-tile autotiles.
        self.convert(tileset=tileset, toExcludeArray=toExcludeArray)

        #Deprecated, exclusion now happens during conversion
        #self.excludeAutotiles(toExcludeArray)

        #Recombine the individual tiles into tilesets
        self.weldOutputs(tileset=tileset)

        #If setAside is true, we'll also combine all the tiles that were excluded into a seperate output
        #This is useful for quickly extracting graphics that were thrown in with Autotiles but which aren't
        #Themselves autotiles.
        if setAside:
            #self.saveAsides(tileset)
            self.weldAside(tileset)
            self.saveAsideWelds(tileset)

        #Save all the finalized tilsets
        self.saveWelds(tileset=tileset)

        #time to clean up
        self.cleanUp()

    #Deprecated
    def excludeAutotiles(self, toExclude):
        if toExclude is None: return
        for i in range(len(toExclude)-1,0-1,-1):
            del self.allAutotiles[toExclude[i]]


    def cleanUp(self):
        ''' Delete cached data. Why do I have cached data on member variables instead of using local variables and return types? Because my functions produce two outputs, not one, the autotiles and everything set aside form and excluded from the auotitles. Is there a better way to do this? sure. Probably. But this worked the first time I wrote it.'''
        if(self.allAutotiles):
            self.allAutotiles.clear()
            self.allAutotiles = None
        if(self.allWelds):
            self.allWelds.clear()
            self.allWelds = None
        if(self.nonAutotiles):
            self.nonAutotiles.clear()
            self.nonAutotiles = None
        if(self.allAsideWelds):
            self.allAsideWelds.clear()
            self.allAsideWelds = None

class TileCompactor:
    '''This tool is used to take multiple tilsets, such as the tilesets produced by TileConverter, and to smash them together into as little space as possible, with a specific output side.'''

    def __init__(self, blockSize:tuple, weldSize:tuple):
        '''This tool is used to take multiple tilsets, such as the tilesets produced by TileConverter, and to smash them together into as little space as possible, with a specific output side.
        1. blockSize: (x,y) in tiles, NOT PIXELS, aka (2,3) or (8,6). What's a block? It's a rectangular chunk of tiles you want to keep together when transfering them to your output file. You commonly want to transfer whole autotiles in a block. (1,1) will ignore blocking and will cram as many tiles as can fit in to whatever space is available. Great for single tiles, not great for keeping autotiles clear and obvious.
        2. weldSize: (x,y) in pixels, NOT TILES, aka (1024,1024) or (2048,2048). Even if you're working with 48x48 tiles, you may still want to output files that have a binary resolution because they play nicer with most graphics cards. 2048x2048 will produce the tightest fit for 48x48.
        '''

        self.blockSize:tuple = blockSize
        '''(x,y) in tiles, NOT PIXELS, aka (2,3) or (8,6)'''

        self.weldSize:tuple = weldSize
        '''(x,y) in pixels, NOT TILES, aka 1024 pixels wide by 1024 pixels tall. Even if you're working with 48x48 tiles, you may still want to output files that have a binary resolution because they play nicer with most graphics cards.'''


    def extractValidBlocks(self, tilesets:List[TileSetInfo]) -> List[Image.Image]:
        '''The purpose of this function is to find out how many actual tiles exist in the tilesets, extract them, and put them in an array of images to make it easier for us to rearrange them and put them back together later. It checks each pixel in each block, and if it finds only transparent pixels, it will not save that block. This allows for you to combine two tilesets with a ton of blank space.'''
        tso = tilesets[0].tileSizeObject

        #Since blockSize is in tiles, let's calculate what the blockSize is in pixels, too.
        blockPixelSizeX = self.blockSize[0] * tso.tileSize
        blockPixelSizeY = self.blockSize[1] * tso.tileSize

        #Rather than trying to keep track of where we are on 2 dimensions in multiple input and output files,
        #we're going to write each block to a temporary buffer in a single directional array
        #and then copy them back out again into the final output
        validBlocks:List[Image.Image] = []
        for m in range(0, len(tilesets)):

            #open the image associated with the tileset.
            tileset = tilesets[m]
            toEdit = Image.open(tileset.loadPath).convert("RGBA")

            #How many blocks can fit in this tileset, horizontally and vertically?
            blocksWide = math.floor(toEdit.size[0]/blockPixelSizeX)
            blocksHigh = math.floor(toEdit.size[1]/blockPixelSizeY)

            #load up the pixels
            px = toEdit.load()

            #We are pasting everything to the same upper left hand corner of each intermediate temporary buffer image
            pastePoint = (0,0)

            #Head through all the blocks
            for h in range(0, blocksHigh):
                for k in range(0, blocksWide):

                    #We're looking for even a single pixel with an alpha that is greater than 0.
                    # If we can find that, it means the block is occupied.
                    foundOpaquePixel = False

                    #Start in the upper left hand corner of the block
                    startX = k * blockPixelSizeX
                    startY = h * blockPixelSizeY

                    #Head through all the pixels of the block
                    for j in range(0, blockPixelSizeY):
                        for i in range(0, blockPixelSizeX):

                            #Analyze the color
                            color = px[i + startX,j + startY]

                            # (R, G, B, A)
                            if color[3] != 0:
                                foundOpaquePixel = True
                                break
                        if foundOpaquePixel:
                            break

                    #If an opaque pixel was found, we add this block to our list
                    if foundOpaquePixel:
                        source = (startX, startY, startX + blockPixelSizeX, startY+ blockPixelSizeY)
                        pastePoint = (0,0)
                        block = Image.new(mode="RGBA", size=(blockPixelSizeX, blockPixelSizeY))
                        block.alpha_composite(im=toEdit,dest=pastePoint, source=source)
                        validBlocks.append(block)

            #Close the file to reduce memory leaks
            toEdit.close()
        #Return our array of images
        return validBlocks

    def reweld(self, tilesets:List[TileSetInfo], validBlocks:List[Image.Image]) -> List[Image.Image]:
        '''Time to recombine the individual tile blocks we saved out during extractValidBlocks into tilesets.
        1. tilesets: A list of all the tilesets to be combined
        2. validBlocks: the output of extractValidBlocks()
        '''
        tso = tilesets[0].tileSizeObject

        #Calculate how big blocks are in pixel size, based on the tileSizeObject of the tilesets
        blockPixelSizeX = self.blockSize[0] * tso.tileSize
        blockPixelSizeY = self.blockSize[1] * tso.tileSize

        #for i in range(0, len(validBlocks)):
        #    for j in range(0, len(validBlocks[i])):
        #        print(validBlocks[i][j])

        #Calculate how many blocks high/wide we can put into the output, based on the weldsize
        blocksWide = math.floor(self.weldSize[0]/blockPixelSizeX)
        blocksHigh = math.floor(self.weldSize[1]/blockPixelSizeY)
        blocksPerWeld = blocksWide*blocksHigh

        #Create an array to hold all our outputs
        totalWelds = math.ceil(len(validBlocks)/blocksPerWeld)
        finalWelds:List[Image.Image] = [None]*totalWelds

        # n will be used to traverse the validBlcoks
        n = 0

        # Go through all the output files we're going to need to make
        for m in range(0, totalWelds):

            #Make the output file
            finalWelds[m] = Image.new(mode="RGBA", size=(self.weldSize[0], self.weldSize[1]))

            #Read through positions where a block can go, and paste in a block
            for j in range(0, blocksHigh):
                for i in range(0, blocksWide):
                    if n>= len(validBlocks):
                        break

                    #All sources are just coming straight from uniformly sized buffer images
                    source = (0,0, blockPixelSizeX, blockPixelSizeY)
                    pastePoint = (i*blockPixelSizeX, j*blockPixelSizeY)

                    #Copy them into the final tilesets
                    finalWelds[m].alpha_composite(im=validBlocks[n], source=source, dest=pastePoint)
                    n +=1

        #return the final tilesets.
        return finalWelds

    def saveWelds(self, finalWelds:List[Image.Image], outputName:str):
        '''Save all the images produced by reweld()
        1. finalWelds: The list of images produced by reweld(), which are finalized tilesets ready to be written to file
        2. outputName: The name you want to attach to the output file. Do not include the file extension. Will have 0,1,2, etc appended
    '''
        n = len(finalWelds)
        for i in range(0, n):
            savePath = outputName + str(i) + ".png"
            finalWelds[i].save(fp=savePath)

    def combineFilesOfSameBlockSize(self, tilesets:List[TileSetInfo], outputName:str):

        #Get all the valid blocks written to a temporary buffer
        validBlocks:List[Image.Image] = self.extractValidBlocks(tilesets)

        finalWelds = self.reweld(tilesets, validBlocks)

        self.saveWelds(finalWelds, outputName)


## Examples
Below are some examples of using the code to transform RTP files (RTP Files not supplied, you have to upload them yourself. I'm using the MV RTP, but it works the same on the MZ one)

In [4]:
# SIMPLEST EXAMPLE
# This will convert RPGMaker RTP autotiles into 47-tile autotiles, and
# output individual files for each autotile

# Create a TileSizeObject. For a standard RPG Maker MV or MZ tileset, this will always be exactly the same.
# You only need to create one, and can reuse it with multiple tilesets, as long as they're all the same size.
tileType = TileSizeObject(32, (0,0), (8,1))

## Create a TileSetInfo. Pass in the name of the file (no file extension) and the TileSizeObject
singleSet1 = TileSetInfo("../src/assets/Graphics/Autotiles/Sea", tileType)

#We'll create a TileSetConverter. It can be reused for multiple conversion operations on multiple TileSetInfos.
converter = TileSetConverter()

#This performs the conversion operation.
converter.convertAndSaveAutotiles(singleSet1)

#Tada! Psst: If you don't see the output on the left panel, notice the left panel
# has a small 'refresh folder button. hit that button. not the browser refresh.

In [14]:
# SIMPLE EXAMPLE
# This will convert the autotiles like above, but will save them all in one
# tileset at the end, instead of individual files

#Same as before
tileType = TileSizeObject(32, (0,0), (8,1))
singleSet1 = TileSetInfo("../src/assets/Graphics/Autotiles/Sea", tileType)

#Here is whre things get different. We need to pass in the size of tileset we want it to
#output to. I find 2048x2048 works the best; most software is optimized for powers of 2
converter = TileSetConverter((2048,256))

#Same as before
converter.convertWeldAndSaveAutotiles(singleSet1)

#Tada!

In [23]:
# NORMAL EXAMPLE
# But wait! The previous examples convert a lot of things that aren't autotiles, like
# waterfalls, and lilly pads. Can we exclude them? Yes we can. Can we get them all
# put in a second tileset so we can use them seperately? Yup, that too!

#Create an exclusion pattern by telling me what 'autotiles' are not really autotiles
# This pattern works for the MV RTP Outside_A1
lotusExclusions = [0,1,2,3,4]

#same as before
tileType = TileSizeObject(32, (0,0), (5,1))
singleSet1 = TileSetInfo("../src/assets/Graphics/Autotiles/Flowers1", tileType)

#I want the excluded tiles output to a separate file for safekeeping, but I don't
#think there will be many of them, so I'm saying I want it to be outputted to a smaller,
#512x512 tileset. The main autotile tileset is still 2048x2048
converter = TileSetConverter((2048,256), (512,512))

#We're going to add the exception in, and 'True' will indicate yes, we do want
#the excluded tiles saved off to the side
converter.convertWeldAndSaveAutotiles(singleSet1, lotusExclusions, True)

#Tada!

In [24]:
## DOUBLE EXAMPLE
# Let's Do two tiles at once, and also modify the suffixes so we can control what
# the output files are named.

# same as before
lotusExclusions = [0,1,2,3,4]
tileType = TileSizeObject(32, (0,0), (5,1))

# two files now, both using the same tileType
singleSet1 = TileSetInfo("../src/assets/Graphics/Autotiles/Flowers1", tileType)
tileType = TileSizeObject(32, (0,0), (8,1))
singleSet2 = TileSetInfo("../src/assets/Graphics/Autotiles/Sea", tileType)

# We've tagged two suffixes on to the end. When output files are made, they'll
# use one of these two suffixes, depending on whether they're autotiles or not
converter = TileSetConverter((2048,512), (512, 512),"_autotile", "_other")

#A1 will use the lotus exclusions, but A2 doesn't have to as there's nothing to exclude
converter.convertWeldAndSaveAutotiles(singleSet1, lotusExclusions, True)
converter.convertWeldAndSaveAutotiles(singleSet2, [], False)

#Tada!

In [45]:
## COMPACTION EXAMPLE
# Wait, but both tilesets we outputted in the previous example... have
# a lot of empty space. Wouldn't it be nice if we could combine them?

# There's an extra tool included that can take any number of files,
# throw away the empty space, and assemble the tiles together on
# a new tileset.


#same as before
lotusExclusions = [0,1,2,3,4]
tileType = TileSizeObject(32, (0,0), (5,1))

# add enough tilesets that saving space will actually result in fewer
# tilesets
singleSet1 = TileSetInfo("../src/assets/Graphics/Autotiles/Flowers1", tileType)
tileType = TileSizeObject(32, (0,0), (8,1))
singleSet2 = TileSetInfo("../src/assets/Graphics/Autotiles/Sea", tileType)

# Same as before
converter = TileSetConverter((2048,256), (256, 256),"_autotile", "_other")

#Send in your files for conversion, and specify their exclusions
converter.convertWeldAndSaveAutotiles(singleSet1, lotusExclusions, True)
converter = TileSetConverter((2048,256), (256, 256),"_autotile", "_other")
converter.convertWeldAndSaveAutotiles(singleSet2, [], False)

#We are now going to read in those files we just made so we can smash them together in as compact a space as possible
weldSet1 = TileSetInfo("../src/assets/Graphics/Autotiles/Flowers1_other0", tileType)
weldSet2 = TileSetInfo("../src/assets/Graphics/Autotiles/Sea_autotile0", tileType)

#Create a new compactor. 8x6 is the size (x, y) in tiles of an autotile, 2048x2048 is
# the output tilesize
compator = TileCompactor((8, 6), (2048,512))

#Perform the compaction operation to stitch together the tilesets. You'll want to send in the tilesets to be combined, AND the
#output file name prefix (it'll add a 0,1,2, etc after this name for each tileset it produces)
compator.combineFilesOfSameBlockSize([weldSet1, weldSet2], "MV_RTP_RW")

#Tada! (Note, there will be a snow tile that looks like its empty, but I assure
# you there is white non-transparent pigment XD)

#Of course you can also create a compactor to work with smaller tile sizes,
#such as (2,3), and combine the 'excluded' tiles as well.





## Your Code
Try one of the examples above, or write your own code!

In [49]:
#Space to Write your code :)
converter = TileSetConverter((2048,256), (256, 256),"_autotile", "_other")

base_path ="../src/assets/Graphics/Autotiles/"
autotiles = ["Sea", "Sea without shore", "Sea deep", "Sand shore",  "Water rock" ]
tileType = TileSizeObject(32, (0,0), (8,1))
setArray = []

for a in autotiles:
    setArray.append(TileSetInfo(base_path+a, tileType))
for set in setArray:
    converter.convertWeldAndSaveAutotiles(set)

# don't autotile these, need to figure out how they're processed
# flowers = "Flowers1"
# lotusExclusions = [0,1,2,3,4]
# tileType = TileSizeObject(32, (0,0), (5,1))
# flowerSet = TileSetInfo(base_path+flowers, tileType)
# converter.convertWeldAndSaveAutotiles(flowerSet, lotusExclusions, True)

# fountain = "Fountain1"
# lotusExclusions = [1,4,7,10,13]
# tileType = TileSizeObject(32, (0,0), (15,1))
# foutainSet = TileSetInfo(base_path+fountain, tileType)
# converter.convertWeldAndSaveAutotiles(foutainSet, lotusExclusions, True)



# Do you want to run this Locally on your Own Machine?
Python is one of those languages where you write very little code to leverage powerful libraries written by someone else in order to automate boring tasks like copying and pasting graphics in a programmatic pattern.

So anyone who wants to work in Python has to get used to the idea that, for every problem they have, there's a library for that. I mean, there's more than one, but there'd definitely *one*, and usually one everyone knows and has made a million tutorials for.

When it comes to quickly editing images, that library is [Pillow](https://pypi.org/project/pillow/). That's what that 'import Image from PIL' line was all about.

If you want to run a verison of this collab on your own computer, you'll need to have a development environment-- I use [Visual Studio Code](https://www.google.com/search?q=visual+studio+code&sca_esv=598988451&rlz=1C1CHZL_enHK710HK710&ei=lCqnZbmZILOt5NoPucKp8AQ&ved=0ahUKEwi5s-OLoOODAxWzFlkFHTlhCk4Q4dUDCBA&uact=5&oq=visual+studio+code&gs_lp=Egxnd3Mtd2l6LXNlcnAiEnZpc3VhbCBzdHVkaW8gY29kZTIQEAAYgAQYigUYQxixAxiDATIKEAAYgAQYigUYQzILEAAYgAQYigUYkQIyCxAAGIAEGIoFGJECMgsQABiABBiKBRiRAjIFEAAYgAQyBRAAGIAEMgUQABiABDIFEAAYgAQyBRAAGIAESOQBUABYAHAAeAGQAQCYAWagAWaqAQMwLjG4AQPIAQD4AQHiAwQYACBB&sclient=gws-wiz-serp) --and to have installed [Python](https://www.python.org/) itself...

You will either need to copy+paste each of the code blocks one by one into your python file or you can run the whole collab locally, like me, by working with [Jupyter Notebooks](https://code.visualstudio.com/docs/datascience/jupyter-notebooks).

You'll need to install something to manage python packages/libraries, if you don't already have one. Options include '[pip](https://colab.research.google.com/drive/1pK-jr_15GjqxKqSUtrmQuLVsSE2_IQq_#scrollTo=D0ukuux044Re&line=14&uniqifier=1)' which I'm pretty sure is straight up bundled with python these days, or a version of '[conda](https://conda.io/projects/conda/en/latest/user-guide/install/index.html).'

The process of setting up python can be briefly overwhelming because it feels like there's a lot of peices just to get started. This can be disheartening if things go wrong and, for example, VSCode can't find your damn Python path variable. Grr! Then you have to set it manually! But if things go wrong on your first attempt, don't give up, you'll get it!
